In [1]:
# Structural Analysis of the disulfide bonds in 2q7q
# Author: Eric G. Suchanek, PhD.
# (c) 2023 Eric G. Suchanek, PhD., All Rights Reserved
# Last Modification: 2/16/23
# Cα N, Cα, Cβ, C', Sγ Å °

import pandas as pd
import numpy

import pyvista as pv
from pyvista import set_plot_theme

from Bio.PDB import *

import proteusPy
from proteusPy import *
from proteusPy.Disulfide import *
from proteusPy.DisulfideList import DisulfideList
from proteusPy.DisulfideLoader import Load_PDB_SS

# pyvista setup for notebooks
pv.set_jupyter_backend('trame')

#set_plot_theme('dark')


# The Most Beautiful Disulfide Bond in the World
Eric G. Suchanek, PhD 2/14/23



### Load the RCSB Disulfide Database
We load the database and get its properties as follows:

In [2]:
PDB_SS = Load_PDB_SS(verbose=True)
PDB_SS.describe()

-> load_PDB_SS(): Reading /Users/egs/repos/proteusPy/proteusPy/data/PDB_SS_ALL_LOADER.pkl... done.
    =========== RCSB Disulfide Database Summary ==============
PDB IDs present:                    35818
Disulfides loaded:                  120697
Average structure resolution:       2.34 Å
Lowest Energy Disulfide:            2q7q_75D_140D
Highest Energy Disulfide:           1toz_456A_467A
Total RAM Used:                     29.26 GB.
    ================= proteusPy: 0.52dev =======================


We see from the statistics above that disulfide 2q7q_75D_140D has the lowest energy, so let's extract it from the database and display it. A few notes about the display window. You might need to click into the window to refresh it. Click drag to rotate the structures, mousewheel to zoom. The window titles display several parameters about the disulfide bonds including their approximate torsional energy, their Ca-Ca distance, and the *torsion length*. The latter parameter is formally, the Euclidean length of the sidechain dihedral angle when treated as a five-dimensional vector. This sounds all mathy and complicated, but in essence it gives a measure of how 'long' that five dimensional vector is. This is used by the DisulfideBond routines to compare individual structures and gauge their structural similarity.

In [3]:
best_ss = PDB_SS['2q7q_75D_140D']
best_ss.pprint()
best_ss.display(style='sb', light=True)

<Disulfide 2q7q_75D_140D, Source: 2q7q, Resolution: 1.6 Å 
Χ1-Χ5: -59.36°, -59.28°, -83.66°, -59.82° -59.91°, 0.49 kcal/mol 
Cα Distance: 5.50 Å 
Torsion length: 145.62 deg>


Widget(value="<iframe src='http://localhost:56424/index.html?ui=P_0x281898f70_0&reconnect=auto' style='width: …

And that, gentle reader, is it. *The most beautiful disulfide bond in the world*! Look at it. It's gorgeous. The sidechain dihdedral angles (Χ1-Χ5: -59.36°, -59.28°, -83.66°, -59.82° -59.91°), and estimated energy, (0.49 kcal/mol) is *lower* than the analytical global minimum, (Χ1-Χ5: -60.00°, -60.00°, -90.00°, -60.00° -60.00°, 0.60 kcal/mol)! Let's have a look at the 'CPK' style rendering to see the structure's overall appearance:


In [ ]:
best_ss.display(style='cpk', light=True)

### Finding Potential Structural Relatives by Sequence Similarity
Working under the assumption that similar sequence -> similar structure I generated a query on: https://www.ebi.ac.uk/pdbe/entry/pdb/2q7q to return PDB IDs for structures with high sequence similarity. This yielded a ```.csv``` file, which we will import below:

In [ ]:
ss_df = pd.read_csv('2q7q_seqsim.csv')
ss_df.head(5)

All of the nearest sequence neighbors are sadly, bacterial. Let's extract the unique ids next.

In [ ]:
relative_list = ss_df['pdb_id'].unique()
relative_list

We now need to convert the list of PDB IDs into real disulfides from the database. We do this with the proteusPy DisulfideLoaderbuild_ss_from_idlist() function. Next we print out some relevant statistics.


In [ ]:
relatives = DisulfideList([], 'relatives')
relatives = PDB_SS.build_ss_from_idlist(relative_list)

print(f'There are: {relatives.length} related structures.\nAverage Energy: {relatives.Average_Energy:.2f} kcal/mol\nAverage Ca distance: {relatives.Average_Distance:.2f} Å')
print(f'Average resolution: {relatives.Average_Resolution:.2f} Å \nAverage torsion distance: {relatives.Average_Torsion_Distance:.2f}°')

In [ ]:
ssmin, ssmax = relatives.minmax_energy()
duolist = DisulfideList([ssmin, ssmax], 'mM')
duolist.display(style='sb', light=True)

In [ ]:
duolist.display_overlay(light=False)

The two Disulfides...

We can find disulfides that are conformationally related by using the DisulfideList.nearest_neighbors() function with a dihedral angle cutoff. This cutoff is measure of angular similarity across all five sidechain dihedral angles.

In [ ]:
close_neighbors = relatives.nearest_neighbors(ssmin, 20.0)


In [ ]:
close_neighbors.length

In [ ]:
close_neighbors.display_overlay(light=False)

In [ ]:
ssTotList = PDB_SS.SSList
global_neighbors = ssTotList.nearest_neighbors(ssmin.chi1, ssmin.chi2, ssmin.chi3, ssmin.chi4, ssmin.chi5, 5.0)
global_neighbors.length

In [ ]:
global_neighbors.display_overlay(light=False)

## References
* *Application of Artificial Intelligence in Protein Design* - Doctoral Dissertation, EG Suchanek, 1987, Johns Hopkins Medical School
* https://doi.org/10.1021/bi00368a023
* https://doi.org/10.1021/bi00368a024
* https://doi.org/10.1016/0092-8674(92)90140-8
* http://dx.doi.org/10.2174/092986708783330566
* https://doi.org/10.1021/bi0603064
* https://doi.org/10.1021/bi9826658
* https://pubmed.ncbi.nlm.nih.gov/22782563/
* 